# Jupyter Notebook for Practical 2: Analysis of an Illumina beadchip microarray experiment in R

### Set the working directory for R and load the packages we will be using

In [ ]:
setwd("/home/h1/538043/limma")
library(limma)
library(statmod)
library(gplots)

### The files we will analyze are: 

Asurgen_desc.txt: A file with descriptions of the samples on the chip. These are four patients from each of which was measured the gene expression in a sample of tumour tissue and a sample of adjacent, healthy tissue. 
TumorAdjacent-Controls.txt: The control probes profile that are informative of the background noise. This could be excess fluorescently labeled cDNA, or other impurities on the chip
Tumor
TumorAdjacent-probe-raw.txt: Contains the detection p-values as well as the intensity of fluorescence obtained by binding of the Oligonucleotide probe on the microchip against the cDNA for each gene and each sample. This file has around 3.2MB. If the experiment had sequenced all cellular mRNA (RNAseq), the file would likely be larger than 1-3GB.   


### Read the input files (can take a few mins)

In [ ]:
x = read.ilmn(files = "TumorAdjacent-probe-raw.txt", ctrlfiles = "TumorAdjacent-Controls.txt")
targets = readTargets("Asurgen_desc.txt")
y = neqc(x)

### Estimate the correlation between duplicate spots (regularly spaced replicate spots on the same array) or between technical replicates from a series of arrays. In this case we want to see if the different replicates (here these are the three different patients) have similar gene expression values

Following Smyth, G.K., Michaud, J. and Scott, H.S., 2005. Use of within-array replicate spots for assessing differential expression in microarray experiments. Bioinformatics, 21(9), pp.2067-2075.

The method involves fitting separate linear models to the expression data for each gene but with a common value for the between-replicate correlation. The method greatly improves the precision with which the genewise variances are estimated and thereby improves inference methods designed to identify differentially expressed genes

In [ ]:
ct = factor(targets$FactorValue)
design = model.matrix(~0+ct)
colnames(design) = levels(ct)
dupcor <- duplicateCorrelation(y,design,block=targets$Patient)

### Combine the results with the "empirical Bayes method" for moderating the genewise variances between genes. This procedure will find the genes differently expressed between tumour and adjacent, correcting for any possible variance that exists within the three individuals. 

Given a microarray linear model fit, compute moderated t-statistics, moderated F-statistic, and log-odds of differential expression by empirical Bayes moderation of the standard errors towards a common value.
The method was described in this paper: 
Smyth, G. K. (2004). Linear models and empirical Bayes methods for assessing differential expression in microarray experiments. Statistical Applications in Genetics and Molecular Biology 3, Article 3. http://www.statsci.org/smyth/pubs/ebayes.pdf


To filter out false positives we use FDR, the false discovery rate: FDR is defined as the expected proportion of Type I errors (false positives) among the rejected hypotheses.


In [ ]:
fit <- lmFit(y,design,block=targets$Donor,correlation=dupcor$consensus.correlation)
contrasts = makeContrasts(Tumor-Adjacent, levels = design)
fit2 = contrasts.fit(fit, contrasts)
fit3 = eBayes(fit2, trend= TRUE)
selected = p.adjust(fit3$p.value, method = "fdr") <0.05
sel = y[selected,]
matrix = data.matrix(sel$E)
rownames(matrix) = sel$gene$SYMBOL


### V for Vendetta: Next we will be generating a "volcano plot" to visualize how many genes are differentially expressed following the eBayes method with FDR of 0.05. The genes differently expressed will be plotted as log Fold change (x) vs. -log10 p-value for the error probability of differential expression (y), and marked blue. The genes identical between the tumour and adjacent tissue samples are depicted in red.

In [ ]:
table = data.frame(topTable(fit3, number = 100000, adjust.method = "fdr"))
table$threshold = as.factor(table$adj.P.Val < 0.05)
library(ggplot2)
ggplot(data = table, aes(x = logFC, y = -log10(adj.P.Val))) +
	geom_point(alpha = 0.4, size = 1.75, aes(colour = threshold)) +
	theme(legend.position = "none")


### As you can see, there are a lot of differences which, given the nature of cancer being a disease that brings the cell cycle and gene expression out of balance, is not too surprising. In order to find out which genes change their expression with highest fold change in each direction (called "overexpressed" and "underexpressed"), we will generate a heatmap from the top 50 differentially expressed genes. The topTable function can help us to do this. The heatmap will show fold changes with respect to the mean fold change of the table. 

In [ ]:
wanted = topTable(fit3, n=50) 
selected2 = rownames(wanted)
esetwant = y[selected2,] 
matrix2 = data.matrix(esetwant$E)
rownames(matrix2) = wanted$SYMBOL
library(gplots)
heatmap.2(matrix2) 

### You can modify your heatmap to show different colours by modifying parameters of the heatmap.2 function. You can add any valid modifiers like this: heatmap.2(matrix2, modifyer=Parameter(NumericalValue)) 

*You can change the colours by adding col = redgreen(), greenred(), bluered(), redblue() or even rainbow(); within the brackets you specify the number of shades of that colour you want
*The cexRow/cexCol modifyer changes the font size of the rows/column labels, eg. cexRow = 0.6  
*trace = “none” gets rid of the density traces on the heatmap. 

Now, generate a new heatmap with a different color scheme, without trace, and with a row fontsize of 0.6 and column fontsize of 1.0. This image is saved on your jupyterhub directory through the dev.print(pdf) command: download it and open in Acrobat or another pdf viewer